In [26]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
#Author: Selvaria
#实时天气每日录入

import pandas
import pymysql
import requests
import concurrent.futures
import time

class WeatherGaode(object):
    
    url = 'https://restapi.amap.com/v3/weather/weatherInfo?parameters'
    key = 'e84aae1ede212a3e043dfd7e8a21cf7e'
    
    def __init__(self, code):
        self.code = code


    def get_data_api(self):  #获取接口传回的原始数据，实时值
        #key = 'e84aae1ede212a3e043dfd7e8a21cf7e'
        params = {
                "key" : self.key, 
                "city" : str(self.code), #需填入城市代码adcode
                "extensions": "base", #base:返回实况天气，all:返回预报天气
                "output" : 'json' 
            }
        data = requests.get(self.url,params = params).json()
        #print(data)
        if data['status'] == '0':
            print(data)
            print('出现错误，极可能是高德地图抽风了，请调试。')
        else:
            return data
        
    def get_data_api_f(self):  #获取接口传回的原始数据，带预测值
        params = {
                "key" : self.key, 
                "city" : str(self.code), 
                "extensions": "all", #all:返回预报天气
                "output" : 'json' 
            }
        data = requests.get(self.url,params = params).json()
        #print(data)
        if data['status'] == '0':
            print(data)
            print('出现错误，极可能是高德地图抽风了，请调试。')
        else:
            return data
        
    def get_weather_detail(self):
        data_origin = self.get_data_api()
        city_status = data_origin['lives']
        if len(city_status[0]) == 0:
            print('%d无数据' %self.code)
            return None
        else:
            report_time = city_status[0]['reporttime'] #数据发布时间
            temp = city_status[0]['temperature'] #温度
            humidity = city_status[0]['humidity'] #湿度
            discribe = city_status[0]['weather'] #天气描述
            wind_d = city_status[0]['winddirection'] #风向
            wind_p = city_status[0]['windpower'] #风力
            province = city_status[0]['province']
            adcode = city_status[0]['adcode']
            data_row = [report_time,temp,humidity,discribe,wind_d,wind_p,province,adcode]
            return data_row
    
    def up_sql_test(self):
        single = self.get_weather_detail_f()
        connection = pymysql.connect(host = '39.105.15.110',user = 'root',password = 'Jusure2016',db = 'hm_data', port=3306, charset='utf8')
        for data in single:
            try :
                with connection.cursor() as cursor:
                    sql_i = "insert into `city_weather_status`(`report_time`,`temperature`,`humidity`,`weather`,`w_direction`,`w_power`,`province`,`adcode`,`forcast`)\
                    values(%s,%s,%s,%s,%s,%s,%s,%s,%s)"
                    cursor.execute(sql_i,data)
                connection.commit()
                #print('%d数据写入成功' %self.code)
            except Exception as e:
                print(str(e))            
        connection.close()
            
    def get_weather_detail_f(self): #带预测值
        data_realtime = self.get_data_api()
        data_forcast = self.get_data_api_f()
        city_status_r = data_realtime['lives']
        city_status_f = data_forcast['forecasts'][0]
        if len(city_status_r[0]) == 0:
            print('%d无数据' %self.code)
            return None
        #else:
        
        #实时天气
        report_time = city_status_r[0]['reporttime'] #数据发布时间
        temp = city_status_r[0]['temperature'] #温度
        humidity = city_status_r[0]['humidity'] #湿度
        discribe = city_status_r[0]['weather'] #天气描述
        wind_d = city_status_r[0]['winddirection'] #风向
        wind_p = city_status_r[0]['windpower'] #风力
        province = city_status_r[0]['province']
        adcode = city_status_r[0]['adcode']
        data_today = [report_time,temp,humidity,discribe,wind_d,wind_p,province,adcode,0]
        
        #预报天气（1天后）     
        if int(report_time[-8:-6]) > 6 and int(report_time[-8:-6]) < 18:
            temp = city_status_f['casts'][1]['daytemp'] #温度
            discribe = city_status_f['casts'][1]['dayweather'] #天气描述
            wind_d = city_status_f['casts'][1]['daywind'] #风向
            wind_p = city_status_f['casts'][1]['daypower'] #风力
        else:
            temp = city_status_f['casts'][1]['nighttemp'] #温度
            discribe = city_status_f['casts'][1]['nightweather'] #天气描述
            wind_d = city_status_f['casts'][1]['nightwind'] #风向
            wind_p = city_status_f['casts'][1]['nightpower'] #风力
        date = city_status_f['casts'][1]['date'] + report_time[-9:]#日期
        
        data_forcast = [date,temp,None,discribe,wind_d,wind_p,province,adcode,1]
        data_row = [data_today, data_forcast]
        return data_row
    
    
def test():
    w = WeatherGaode(110111)
    #w.get_data_api()
    #x = w.get_weather_detail_f()
    #print(x)
    w.up_sql_test()

def city_table(): #获取城市代码列表
    connection = pymysql.connect(host = '39.105.15.110',user = 'root',password = 'Jusure2016',db = 'hm_data', port=3306, charset='utf8')
    sql = "SELECT * FROM `citycode_gaode`"
    city_df = pandas.read_sql(sql,connection) 
    connection.close()
    return city_df

def city_code_list():
    connection = pymysql.connect(host = '39.105.15.110',user = 'root',password = 'Jusure2016',db = 'hm_data', port=3306, charset='utf8')
    with connection.cursor() as cursor: 
        sql = "SELECT * FROM `gaode_address_code`"
        cursor.execute(sql)
        code_form = cursor.fetchall()
    connection.close()
    code_list = []
    for i in range(len(code_form)):
        code = code_form[i][3]
        code_list.append(code)
    return code_list

def _get_weather_status(code):
    w = WeatherGaode(code)
    try:
        w.up_sql_test()
        #print('%s is Done!' %code)
    except Exception as e:
        print(str(e))


def main():
    code_list = city_code_list()
    with concurrent.futures.ThreadPoolExecutor(40) as executor:
        for code, result in zip(code_list, executor.map(_get_weather_status, code_list)):
            print('%s is Done!' %code)
    print('All Done!')
    
    
if __name__ == '__main__':
    startTime = time.time()
    #main()
    test()
    endTime = time.time()
    secs = (endTime - startTime)
    print('用时:',secs)


OperationalError: (1045, "Access denied for user 'root'@'10.0.42.1' (using password: NO)")

In [6]:
#这个cell是整理城市csv文件到数据库
df = pandas.read_excel('./citycode.xlsx')
df_p = df[df.isnull().citycode==False]
df_p.head()
#df_p.to_csv('./citycode.csv', encoding = 'gb18030')

,中文名,adcode,citycode
1,北京市,110000,10.0
2,北京市市辖区,110100,10.0
3,东城区,110101,10.0
4,西城区,110102,10.0
5,朝阳区,110105,10.0


In [1]:
import pandas
import pymysql
import requests


url = 'https://restapi.amap.com/v3/weather/weatherInfo?parameters'
key = 'e84aae1ede212a3e043dfd7e8a21cf7e'

params = {
        "key" : key, 
        "city" : 110100, #需填入城市代码adcode
        "extensions": "all", #base:返回实况天气，all:返回预报天气
        "output" : 'json' 
    }
data = requests.get(url,params = params).json()
#print(data)
if data['status'] == '0':
    print(data)
    print('出现错误，极可能是高德地图抽风了，请调试。')
else:
    print(data)

{'status': '1', 'count': '1', 'info': 'OK', 'infocode': '10000', 'forecasts': [{'city': '北京城区', 'adcode': '110100', 'province': '北京', 'reporttime': '2019-03-13 08:22:04', 'casts': [{'date': '2019-03-13', 'week': '3', 'dayweather': '多云', 'nightweather': '多云', 'daytemp': '16', 'nighttemp': '2', 'daywind': '西北', 'nightwind': '西北', 'daypower': '4', 'nightpower': '4'}, {'date': '2019-03-14', 'week': '4', 'dayweather': '多云', 'nightweather': '多云', 'daytemp': '15', 'nighttemp': '4', 'daywind': '西北', 'nightwind': '西北', 'daypower': '4', 'nightpower': '4'}, {'date': '2019-03-15', 'week': '5', 'dayweather': '多云', 'nightweather': '多云', 'daytemp': '12', 'nighttemp': '0', 'daywind': '北', 'nightwind': '北', 'daypower': '4', 'nightpower': '4'}, {'date': '2019-03-16', 'week': '6', 'dayweather': '晴', 'nightweather': '晴', 'daytemp': '16', 'nighttemp': '1', 'daywind': '北', 'nightwind': '北', 'daypower': '4', 'nightpower': '4'}]}]}


In [2]:
data

{'count': '1',
 'forecasts': [{'adcode': '110100',
   'casts': [{'date': '2019-03-13',
     'daypower': '4',
     'daytemp': '16',
     'dayweather': '多云',
     'daywind': '西北',
     'nightpower': '4',
     'nighttemp': '2',
     'nightweather': '多云',
     'nightwind': '西北',
     'week': '3'},
    {'date': '2019-03-14',
     'daypower': '4',
     'daytemp': '15',
     'dayweather': '多云',
     'daywind': '西北',
     'nightpower': '4',
     'nighttemp': '4',
     'nightweather': '多云',
     'nightwind': '西北',
     'week': '4'},
    {'date': '2019-03-15',
     'daypower': '4',
     'daytemp': '12',
     'dayweather': '多云',
     'daywind': '北',
     'nightpower': '4',
     'nighttemp': '0',
     'nightweather': '多云',
     'nightwind': '北',
     'week': '5'},
    {'date': '2019-03-16',
     'daypower': '4',
     'daytemp': '16',
     'dayweather': '晴',
     'daywind': '北',
     'nightpower': '4',
     'nighttemp': '1',
     'nightweather': '晴',
     'nightwind': '北',
     'week': '6'}],
   'c

In [15]:
time = data['forecasts'][0]['reporttime']
int(time[-8:-6])

8

In [6]:
import pandas
import pymysql

connection = pymysql.connect(host = '39.105.15.110',user = 'root',password = 'Jusure2016',db = 'hm_data', port=3306, charset='utf8')
with connection.cursor() as cursor: 
    sql = "SELECT * FROM `citycode_gaode`"
    cursor.execute(sql)
    code_form = cursor.fetchall()
connection.close()

for i in range(len(code_form)):
    code = code_form[0][2]
    print(code)

110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000

110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000
110000

In [1]:
import pymysql

connection = pymysql.connect(host = '39.105.15.110', user = 'root', passwd ='Jusure2016',db = 'hm_data', charset='utf8')